In [1]:
# Imports
import pandas as pd
import numpy as np
import nltk
import string
import re
from pickle import dump, load

## Preprocessing

In [2]:
url = "https://uom-twitter-sentiment-analysis.s3.us-east-2.amazonaws.com/Twitter_sentiment_clean.csv"
data = pd.read_csv(url)

In [ ]:
data.head(50)

In [31]:
# Remove punctuation
def remove_punct(text):
    new_text = []
    for t in text:
        if t not in string.punctuation:
            new_text.append(t)
    return ''.join(new_text)

In [32]:
# Tokenizer
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [33]:
# Removing stop words
from nltk.corpus import stopwords

def remove_sw(text):
    new_text = []
    for t in text:
        if t not in stopwords.words('english'):
            new_text.append(t)
    return new_text

In [34]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    new_text = []
    for t in text:
        lem_text = lemmatizer.lemmatize(t)
        new_text.append(lem_text)
    return new_text

In [3]:
df_1 = data.iloc[:10000,:]
df_1.head(50)

,target,text
0,0,AHHH I HOPE YOUR OK!!!
1,0,"cool , i have no tweet apps for my razr 2"
2,0,i know just family drama. its lame.hey next ...
3,0,School email won't open and I have geography ...
4,0,upper airways problem
5,0,Going to miss Pastor's sermon on Faith...
6,1,on lunch....dj should come eat with me
7,0,oh why are you feeling like that?
8,0,gahh noo!peyton needs to live!this is horrible
9,1,thank you glad you like it! There is a produc...


In [ ]:
df_1['text'] = df_1['text'].apply(lambda t: remove_punct(t))
df_1['text'] = df_1['text'].apply(lambda t: tokenizer.tokenize(t.lower()))
df_1['text'] = df_1['text'].apply(lambda t: remove_sw(t))

In [ ]:
data['text'] = data['text'].apply(lambda t: remove_punct(t))
data['text'] = data['text'].apply(lambda t: tokenizer.tokenize(t.lower()))
data['text'] = data['text'].apply(lambda t: remove_sw(t))

In [ ]:
data['text'] = data['text'].apply(lambda t: word_lemmatizer(t))

In [ ]:
df_1['text']

## Split Cleaned Data into Training Set and Testing Set

In [6]:
# Read preprocessed csv
url = "https://uom-twitter-sentiment-analysis.s3.us-east-2.amazonaws.com/Lemmatize.csv"
lemmatized_df = pd.read_csv(url)
lemmatized_df.head()

,target,text
0,0,"['ahhh', 'hope', 'ok']"
1,0,"['cool', 'tweet', 'apps', 'razr', '2']"
2,0,"['know', 'family', 'drama', 'lamehey', 'next',..."
3,0,"['school', 'email', 'wont', 'open', 'geography..."
4,0,"['upper', 'airway', 'problem']"


In [9]:
X = lemmatized_df['text']
y = lemmatized_df['target']

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

TfidV = TfidfVectorizer()
X = TfidV.fit_transform(X)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

## Multinomial Naive Bayes Model

In [12]:
# Multinomial Naive Bayes Model
from sklearn.naive_bayes import MultinomialNB

nbc = MultinomialNB()
nbc.fit(X_train, y_train)

MultinomialNB()

In [13]:
y_predict_nbc = nbc.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix, f1_score

cm_nbc = confusion_matrix(y_test, y_predict_nbc)

f1_nbc = f1_score(y_test, y_predict_nbc)
print(f'F1 Score: {f1_nbc}')

training_score = nbc.score(X_train, y_train)
print(f'Training Score: {training_score}')

testing_score = nbc.score(X_test, y_test)
print(f'Testing Score: {testing_score}')

F1 Score: 0.760176194374941
Training Score: 0.8103716666666667
Testing Score: 0.7649325


In [9]:
# Print confusion matrix
cm_nbc

array([[156953,  43020],
       [ 51007, 149020]])

In [10]:
# Testing nbc model for best parameters
from sklearn.model_selection import GridSearchCV
parameters = {'alpha': [1, 0.1, 0.001, 0.0001, 0.00001, 0.000001], 'fit_prior' :[True, False]}
gs_nbc = GridSearchCV(nbc, parameters, n_jobs = -1)
gs_nbc = gs_nbc.fit(X_train, y_train)

In [11]:
gs_nbc.best_score_

0.76318

In [12]:
gs_nbc.best_params_

{'alpha': 1, 'fit_prior': True}

## SGD Model

In [13]:
# SGD Model
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(alpha = 1e-06, penalty = 'l2')
clf.fit(X_train, y_train)

SGDClassifier(alpha=1e-06)

In [14]:
y_predict_clf = clf.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix, f1_score

cm_clf = confusion_matrix(y_test, y_predict_clf)

f1_clf = f1_score(y_test, y_predict_clf)
print(f'F1 Score: {f1_clf}')

training_score = clf.score(X_train, y_train)
print(f'Training Score: {training_score}')

testing_score = clf.score(X_test, y_test)
print(f'Testing Score: {testing_score}')

F1 Score: 0.7877500416531907
Training Score: 0.8198575
Testing Score: 0.783435


In [ ]:
# Testing clf model for best parameters
from sklearn.model_selection import GridSearchCV
parameters = {'alpha': [1, 0.1, 0.001, 0.0001, 0.00001, 0.000001], 'fit_prior' :[True, False], 'class_prior': ['optimal'], 'loss': ['log_loss', 'hinge', 'perceptron', 'modified_huber']}
gs_clf = GridSearchCV(nbc, parameters, n_jobs = -1)
gs_clf = gs_clf.fit(X_train, y_train)

In [ ]:
gs_clf.best_score_

In [ ]:
gs_clf.best_params_

## Logistic Model

In [19]:
# Logistic Model
from sklearn.linear_model import LogisticRegression

lrc = LogisticRegression()
lrc.fit(X_train, y_train)

/Users/brandongroenewold/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [20]:
y_predict_lrc = lrc.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix, f1_score

cm_lrc = confusion_matrix(y_test, y_predict_lrc)

f1_lrc = f1_score(y_test, y_predict_lrc)
print(f'F1 Score: {f1_lrc}')

training_score = lrc.score(X_train, y_train)
print(f'Training Score: {training_score}')

testing_score = lrc.score(X_test, y_test)
print(f'Testing Score: {testing_score}')

F1 Score: 0.7883377878540611
Training Score: 0.7958616666666667
Testing Score: 0.783405


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {"penalty":["l1","l2"], 'C': np.logspace(-3,3,7), 'solver'  : ['newton-cg', 'lbfgs', 'liblinear']}
gs_lrc = GridSearchCV(lrc, parameters)
gs_lrc.fit(X_train, y_train)

In [ ]:
gs_lrc.best_score_

In [ ]:
gs_lrc.best_params_

## LGB Model

In [19]:
# LGB Model
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(scale_pos_weight = 3)
lgb.fit(X_train, y_train)

LGBMClassifier(scale_pos_weight=3)

In [20]:
y_predict_lgb = lgb.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix, f1_score

cm_lgb = confusion_matrix(y_test, y_predict_lgb)

f1_lgb = f1_score(y_test, y_predict_lgb)
print(f'F1 Score: {f1_lgb}')

training_score = lgb.score(X_train, y_train)
print(f'Training Score: {training_score}')

testing_score = lgb.score(X_test, y_test)
print(f'Testing Score: {testing_score}')

F1 Score: 0.7380586789454163
Training Score: 0.6591325
Testing Score: 0.6592425


In [ ]:
# Testing lgb model for best parameters
from sklearn.model_selection import GridSearchCV
parameters = {'num_leaves': [32, 31, 30, 29, 28], 'max_depth' :[-1, -2, -3, -4, -5, -6], 'n_estimators': [100, 200, 500, 1000], 'learning_rate': [.1, .25, .5, .01, .001]}
gs_lgb = GridSearchCV(lgb, parameters, n_jobs=-1, verbose=2)
gs_lgb = gs_lgb.fit(X_train, y_train)

## Saving the Vectorizer and Models

In [22]:
# Save the scaling function to a pickle file (i.e., "pickle it")
# so we can use it from the Flask server. 
print('Saving the vectorizer')
dump(TfidV, open('TfidV.pkl', 'wb'))

Saving the vectorizer


In [23]:
# Save the model to a pickle file (i.e., "pickle it")
# so we can use it from the Flask server. 
print('Saving the model')
dump(clf, open('clf.pkl', 'wb'))

Saving the model


In [24]:
# Save the model to a pickle file (i.e., "pickle it")
# so we can use it from the Flask server. 
print('Saving the model')
dump(lrc, open('lrc.pkl', 'wb'))

Saving the model


## Predictions

In [25]:
# Define prediction labels.
predict_labels = ['Negative', 'Positive']

KeyboardInterrupt: 

In [26]:
# Load the model.
clf1 = load(open('clf.pkl', 'rb'))

In [27]:
lrc1 = load(open('lrc.pkl', 'rb'))

In [28]:
# Load the vectorizer.
TfidV1 = load(open('TfidV.pkl', 'rb'))

In [49]:
# 1. Create three diffferent sets of inputs (i.e., three
# different irises). Note that each set is constructed 
# as a list inside of another list (or an array inside of
# another array). This is how scikit-learn needs it. 
input_row = ['Minnesota drivers are fantastic!']

In [50]:
#2. Tokenizing etc
input_row = remove_punct(input_row)
input_row = tokenizer.tokenize(input_row.lower())
input_row = remove_sw(input_row)
input_row = word_lemmatizer(input_row)

In [51]:
# 3. Transform each input using the scaler function.
input_row_vectorized = TfidV1.transform(input_row)

In [52]:
# 4. Make a prediction for each input.
print('Making predictions: ')
predict = lrc1.predict(input_row1_vectorized)
print(f'> Prediction: {predict_labels[predict[0]]}')

Making predictions: 
> Prediction: Positive


In [53]:
prob = np.round(lrc1.predict_proba(input_row_vectorized)[0], 2) * 100
print(f'This tweet is predicted to be {prob[0]}% negative.')
print(f'This tweet is predicted to be {prob[1]}% positive.')

This tweet is predicted to be 41.0% negative.
This tweet is predicted to be 59.0% positive.


In [48]:
print('Finished')

Finished
